In [14]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np

In [15]:
classes = ['kitakata', 'sano']

batch_size = 32
nb_classes = len(classes)

img_rows, img_cols = 150, 150
channels = 3

train_data_dir = 'train_images'
validation_data_dir = 'test_images'

nb_train_samples = 500
nb_val_samples = 50
nb_epoch = 10

steps_per_epoch =  int(nb_train_samples/batch_size)

In [16]:
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [17]:
# VGG16モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

In [18]:
# 全結合層を構築
# Flattenへの入力指定はバッチ数を除く
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

In [19]:
# VGG16と全結合層を接続
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

# 最後のconv層の直前までの層は、学習済のため、学習させない
for layer in model.layers[:15]:
    layer.trainable = False

optimizer_adam = optimizers.Adam(lr=1e-4)
optimizer_sgd = optimizers.SGD(lr=1e-4, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer_adam,
              metrics=['accuracy'])

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

 # サブディレクトリをクラス名にしておくと、テスト用の画像を生成してくれる
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 998 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [8]:
# Fine-tuning
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=nb_epoch,
    validation_data=validation_generator,
    validation_steps=nb_val_samples)

Epoch 1/10
15/15 [==============================] - 262s 17s/step - loss: 0.8646 - acc: 0.5021 - val_loss: 0.6577 - val_acc: 0.5712
Epoch 2/10
15/15 [==============================] - 224s 15s/step - loss: 0.6552 - acc: 0.6024 - val_loss: 0.5886 - val_acc: 0.7002
Epoch 3/10
15/15 [==============================] - 271s 18s/step - loss: 0.5981 - acc: 0.6979 - val_loss: 0.5054 - val_acc: 0.7595
Epoch 4/10
15/15 [==============================] - 249s 17s/step - loss: 0.5124 - acc: 0.7345 - val_loss: 0.4947 - val_acc: 0.7508
Epoch 5/10
15/15 [==============================] - 243s 16s/step - loss: 0.5006 - acc: 0.7468 - val_loss: 0.4880 - val_acc: 0.7310
Epoch 6/10
15/15 [==============================] - 236s 16s/step - loss: 0.4439 - acc: 0.7917 - val_loss: 0.4722 - val_acc: 0.7793
Epoch 7/10
15/15 [==============================] - 237s 16s/step - loss: 0.3856 - acc: 0.8229 - val_loss: 0.7125 - val_acc: 0.6590
Epoch 8/10
15/15 [==============================] - 473s 32s/step - loss: 0.

In [9]:
model.save_weights(os.path.join(result_dir, 'finetuning.h5'))